In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# import cv2 as cv
# import os
# import numpy as np
# import tensorflow as tf
# import matplotlib.pyplot as plt
# import mediapipe
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

* Module For loading faces from files then extract faces also resize images 
--> in which face net model recive images with targetsize ->> (160, 160) 

**EXTRACTING Faces embdinges** 

In [3]:
# from keras_facenet import FaceNet
# embedder = FaceNet()

# def get_embedding(face_img):
#     face_img = face_img.astype('float32') # 3D(160x160x3)
#     face_img = np.expand_dims(face_img, axis=0) 
#     # 4D (Nonex160x160x3)
#     yhat= embedder.embeddings(face_img)
#     return yhat[0] # 512D image (1x1x512)

In [4]:
# import pickle  
# # import cv2 as cv
# # import os
# # import numpy as np
# # import tensorflow as tf
# # import matplotlib.pyplot as plt
# # import mediapipe
# pickle_modle = pickle.load(open("model_2.pkl", "rb"))

In [5]:
# # y = np.load("faces_embeddings_done_4classes.npz") 
# nn = np.load("faces_embeddings_done_4classes.npz")
# # npzfile.files
# y = nn['arr_1']
# y



array(['sly', 'sly', 'sly', 'sly', 'sly', 'sly', 'sly', 'sly', 'sly',
       'sly', 'abd', 'abd', 'abd', 'abd', 'abd', 'cr7', 'cr7', 'cr7',
       'cr7', 'cr7', 'cr7', 'messi', 'messi', 'messi', 'messi', 'messi',
       'messi', 'messi'], dtype='<U5')

In [6]:
# from sklearn.preprocessing import LabelEncoder

# encoder = LabelEncoder()
# encoder.fit(y)
# y = encoder.transform(y)
# y


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2])

In [6]:
import mediapipe 
import cv2
import matplotlib.pyplot as plt
import cv2 as cv
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import mediapipe
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_img):
    face_img = face_img.astype('float32') 
    face_img = np.expand_dims(face_img, axis=0) 
    yhat= embedder.embeddings(face_img)
    return yhat[0] 


import pickle  
pickle_modle = pickle.load(open("model_2.pkl", "rb"))

nn = np.load("faces_embeddings_done_4classes.npz")
y = nn['arr_1']


from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)



cam = cv2.VideoCapture(0)

mediapipe_drawing = mediapipe.solutions.drawing_utils
mediapipe_facemesh = mediapipe.solutions.face_mesh
face_mesh = mediapipe_facemesh.FaceMesh()
face_detector_model = mediapipe.solutions.face_detection
face_detector = face_detector_model.FaceDetection(min_detection_confidence=0.6)
draw_specs = mediapipe_drawing.DrawingSpec(thickness = 1 ,circle_radius = 2 )


save_dir = "Desktop/images/"
image_counter = 0 

while True :
    timer = cv2.getTickCount()
    sucsses , img = cam.read()
    image_RGB = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    RESULT = face_mesh.process(image_RGB)
    reusult2 = face_detector.process(image_RGB) 
    if RESULT.multi_face_landmarks:
        if reusult2.detections:
            
            for facelanmark  in RESULT.multi_face_landmarks:
                
                

                for id,lm in enumerate(facelanmark.landmark):
                    ih , iw , ic = img.shape
                    x , y = (int(lm.x*iw) , int(lm.y*ih))
                for face_detected in reusult2.detections:
                       ih , iw , ic = img.shape
                       bounding_box = face_detected.location_data.relative_bounding_box
                       

                       x = int(bounding_box.xmin * iw)
                       w = int(bounding_box.width * iw)
                       y = int(bounding_box.ymin * ih)
                       h = int(bounding_box.height * ih)
                       
#cv2.rectangle(img,(x,y),(x +w , y+h ),color = (255,0,0),thickness = 3 )
                       CROPD_IMG = img[y:y+h , x:x+w]
                       test_im = get_embedding(CROPD_IMG)

                       image = cv2.rectangle(img,(x,y),(x +w , y+h ),color = (255,0,0),thickness = 3 )
                       test_im = [test_im]
#                        t_im = cv2.resize(t_im, (160,160))
                       ypreds = pickle_modle.predict(test_im)
                       
                        
                       if encoder.classes_[ypreds[0]] in y:
                            name = encoder.classes_[ypreds[0]]
                            cv2.putText(img=image, text=name, org=(x, y-10), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=3, color=(0, 255, 0),thickness=1)
                       else:
                            cv2.putText(img=image, text="Person not known", org=(x, y-10), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=3, color=(0, 0, 255),thickness=1)

    
    
    
#                        image = cv2.rectangle(img,(x,y),(x +w , y+h ),color = (255,0,0),thickness = 3 )
#                        cv2.putText(img=image, text=encoder.inverse_transform(ypreds)[0], org=(x, y-10), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=3, color=(0, 255, 0),thickness=1)
    
         
    
#                        if image_counter == 2000 : 
#                             break
        
#                        cv2.imwrite(save_dir + "crop_img_{}.png".format(image_counter), CROPD_IMG)
#                        image_counter +=1
                        
                        

    #             another for loop to get number of all land markas to get deep 
    #             knowing start and edges of land marks 
            
                
#                 cropd_images = img[y:y+ih - 500  , x:x+iw-500]
                
#                 furure more need to save all in a list 
#                 which will be done in a module will be much faster 
                
    # calculating frames 
#     Ctime = time.time()
    # fps 
    # current_time - previos_time 
#     fps = 1/(Ctime - Ptime)
    fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer)
#     Ptime = Ctime
    cv2.putText(img =image, text=encoder.inverse_transform(ypreds)[0], org=(x, y-10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255),thickness=2)
   
    # SHOWING IMAGE FROM CAPTURED CAM 
    cv2.imshow('ORGINAL IMAGE' , img) 
    cv2.imshow('croped IMAGE' , CROPD_IMG)  
#     cv2.imshow('crop IMAGE' , cropd_images)
    # ESC | EXIT FROM THE FRAME SHOUTDOWN CAM 
#     if cv2.waitKey(1) == 27:
#         break 
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
         break
   
cam.release()
cv2.destoryAllWindows()

    

1/1 [==============================] - 2s 2s/step


TypeError: argument of type 'int' is not iterable